In [6]:
from pyspark.sql import SparkSession

scala_version = '2.13'
spark_version = '3.5.5'
packages = [ f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}','org.apache.kafka:kafka-clients:2.10.0']
spark = SparkSession.builder.master('local')\
.appName("kafka-example").config("spark.jars.packages", ",".join(packages))\
.config("spark.memory.offHeap.enabled","true")\
.config("spark.memory.offHeap.size","10g")\
.config("spark.driver.memory", "16g")\
.getOrCreate()

In [7]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [8]:
topic_name = 'RandomNumberStreaming'
kafka_server = 'localhost:9092'
kafkaDf = spark.read.format("kafka")\
.option("kafka.bootstrap.servers", "localhost:9092")\
.option("subscribe", topic_name)\
.option("startingOffsets", "earliest")\
.load()

In [9]:
from pyspark.sql.functions import col
batchDF = kafkaDf.select(col('topic'),col('offset'),col('value').cast('string').alias('rand_number'))

In [11]:
from time import sleep
from IPython.display import display, clear_output
for x in range(0, 100):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"Seconds passed: {x*5}")
        display(batchDF.toPandas())
        sleep(5)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break
print("Live view ended...")


Showing live view refreshed every 5 seconds
Seconds passed: 355


,topic,offset,rand_number
0,RandomNumberStreaming,0,"""65"""
1,RandomNumberStreaming,1,"""54"""
2,RandomNumberStreaming,2,"""91"""
3,RandomNumberStreaming,3,"""12"""
4,RandomNumberStreaming,4,"""88"""
...,...,...,...
250,RandomNumberStreaming,250,"""80"""
251,RandomNumberStreaming,251,"""52"""
252,RandomNumberStreaming,252,"""75"""
253,RandomNumberStreaming,253,"""96"""


break
Live view ended...
